In [15]:
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch import optim

feature_size = 3
timesteps = 250
positions = ['layingdown','sitting','standing','walking']

In [60]:
X = []
y = []
for ip, p in enumerate(positions):
    data = []
    with open('data/train/'+p+'/1_android.sensor.accelerometer.data.csv') as f:
        row = []
        for i, line in enumerate(f):
            row.append([float(x) for x in line.split(',')[1:4]])
            if len(row) == timesteps:
                data.append(np.stack(row))
                row = []
    
#     data = np.swapaxes(np.stack(data), 1, 2)
    data = np.stack(data)
    X.append(data)
    y.append(np.zeros(data.shape[0])+ip)

X = np.vstack(X)
y = np.concatenate(y)
print(X.shape, y.shape)


(5807, 250, 3) (5807,)


In [61]:
# print(X.shape[0])
# arr = np.arange(X.shape[0])
# np.random.shuffle(arr)
# print(arr)
# batch_size = 32
# batches, batch_x, batch_y = [], [], []
# for a, i in enumerate(arr):
#     batch_x.append(X[i])
#     batch_y.append(y[i])
#     if a == len(arr)-1 or len(batch_x) == batch_size:
#         batches.append((batch_x, batch_y))
#         batch_x, batch_y = [], []
def dataloader(train_dataset, batch_size):
    # for i, (images, labels) in enumerate(train_loader)
    X, y = train_dataset
    arr = np.arange(X.shape[0])
    print(arr)
    np.random.shuffle(arr)
    batches, batch_x, batch_y = [], [], []
    for a, i in enumerate(arr):
        batch_x.append(X[i])
        batch_y.append(y[i])
        if a == len(arr)-1 or len(batch_x) == batch_size:
            print((np.stack(batch_x).shape, np.array(batch_y).shape))
            batches.append((np.stack(batch_x), np.array(batch_y)))
            batch_x, batch_y = [], []
    return batches

batches = dataloader((X,y), 32)

[   0    1    2 ... 5804 5805 5806]
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32,))
((32, 250, 3), (32

In [20]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable


# Hyper Parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

# MNIST Dataset
train_dataset = dsets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

rnn = RNN(input_size, hidden_size, num_layers, num_classes)


# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, sequence_length, input_size))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        print(images.size())
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size))
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total)) 

# Save the Model
torch.save(rnn.state_dict(), 'rnn.pkl')

torch.Size([100, 28, 28])
torch.Size([100, 28, 28])
torch.Size([100, 28, 28])
torch.Size([100, 28, 28])
torch.Size([100, 28, 28])
torch.Size([100, 28, 28])
torch.Size([100, 28, 28])
torch.Size([100, 28, 28])
torch.Size([100, 28, 28])
torch.Size([100, 28, 28])
torch.Size([100, 28, 28])
torch.Size([100, 28, 28])
torch.Size([100, 28, 28])


KeyboardInterrupt: 